In [ ]:
import os

MODEL = 'llama2'

file_path = os.path.join(os.path.dirname(__file__), r'src\databases\Bizuario Geral.docx')

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

model.invoke('Me conte uma piada')


In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
# Adaptar para ler docx ou converter para pdf?

loader = Docx2txtLoader(file_path)
bizuario_doc = loader.load_and_split()
bizuario_doc

In [ ]:
from langchain.prompts import PromptTemplate

# Prompt (System Message. Define Tasks, Tone, Behaviour and Safety params)
template = '''
Você é um assistente virtual de uma área de materiais chamada Spare Parts Planning.
Sua função será responder à questões genéricas feitas pelos colaboradores da área.
Vou lhe passar um documento com diversas informações relevantes, tais como significado de siglas, telefones úteis, explicação de processos, etc, para que você use como referência para responder ao questionamento do usuário.

Siga todas as regras abaixo:
1/ Você deve buscar se comportar de maneira sempre cordial e solícita para atender aos questionamentos dos usuários.

2/ Algumas linhas do documento fornecido podem conter informações irrelevantes. Preste atenção ao conteúdo útil da mensagem.

3/ Existem informações pessoais dos colaboradores no documento, tais como número de telefone, evite passá-las sob quaisquer circunstâncias.
A única informação que pode ser passada relacionada aos colaboradores é o respectivo login.

4/ Em hipótese alguma envolva-se em discussões de cunho pessoal, sobre tecer opiniões sobre um colaborador ou outro. Caso a pergunta seja neste sentido, recuse-se gentilmente a opinar e ofereça ao usuário ajuda nas questões relevantes.

Aqui está uma pergunta recebida de um usuário.
{question}

Aqui está o documento com as informações relevantes mencionadas.
Esse arquivo servirá de base para que você compreenda nosso contexto de negócio, organização e nossas siglas mais comumente utilizadas.
{bizuario_document}

Escreva a melhor resposta que atende ao questionamento do usuário:
'''

prompt = PromptTemplate.from_template(template)
print(prompt.format(context='Here is some context', question='Here is a question'))

In [ ]:
chain.input_schema.schema()

In [ ]:
chain = prompt | model

chain.invoke(
    {
    'context': 'O nome que me foi dado é Paulo',
    'question': 'Qual o meu nome?'
    }
)

In [ ]:
# In this part, VectorStore will be saved in memory only for concluding goals purpose. When developing the app, a fixed store should
# be implemented. Such as sqlite, postgresql or something like that
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(bizuario_doc, embedding=embeddings)


In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke('O que significa EPEP?')

In [ ]:
# Running
from operator import itemgetter

chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question')
    }
    | prompt
    | model
)

chain.invoke({'question': 'O que significa EPEP?'})

In [ ]:
#pip install docarray
#pydantic==1.10.8
